In [ ]:
from component import tile
from component import model

In [ ]:
# # for debug
# import pandas as pd
# import geopandas as gpd
# from component import parameter as cp

# tb_model = model.TableModel()
# tb_model.json_table = '{"pathname": "/home/prambaud/modules/clip-time-series/utils/clip_test_points.csv", "id_column": "id", "lat_column": "Y", "lng_column": "X"}'


# df = pd.read_csv(cp.test_dataset, sep=None, engine='python')
# df = df.filter(items=["lat", "lng", "id"])
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs='EPSG:4326')
# tb_model.pts = gdf

In [ ]:
viz_model = model.VizModel()

In [ ]:
viz_input_tile = tile.InputTile(viz_model, tb_model)

In [ ]:
viz_input_tile